In [1]:
# import pandas as pd
# from pathlib import Path
# from tqdm import tqdm

In [2]:
# from transformers import pipeline

2023-03-23 10:28:46.963099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [115]:
# Load three datasets: dataset contains all the data without labels, dataset_train_eval contains data with labels
# test contains data without labels
# Need to create train, test and eval
from datasets import load_dataset
dataset = load_dataset("csv",  data_files="../indexing/output.csv")
# We split the data test:eval 70:30 
dataset_train_eval = (load_dataset("csv", data_files="./train.csv", split = "train").train_test_split(test_size=0.3))
test = load_dataset("csv", data_files="./test.csv")
# dataset["train"][100]

Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-df29f072a3b1628c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-baf29a52fe593227/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-fe9974d2b3317d75/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [164]:
# Rename the column label to label_fine_grained [0,5]
dataset_train_eval = dataset_train_eval.rename_column("label", "label_fine_grained")

In [165]:
# Rename the column label_posneg to label [0, 2]
dataset_train_eval = dataset_train_eval.rename_column("label_posneg", "label")

In [166]:
# Check the dataset_train_eval
dataset_train_eval["test"]

Dataset({
    features: ['Initial Index', 'DATE', 'AUTHOR', 'TEXT', 'FAVORITE', 'year', 'manufacturer', 'model', 'label', 'label_posneg'],
    num_rows: 720
})

In [167]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [168]:
# Tokenizer: Hello, How are You: [Hello, How, Are, You]
# Specify Max Length 42 for now, the default model does not have a predefined maximum length
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)


def tokenize_function(examples):
    return tokenizer(examples["TEXT"], padding="max_length", truncation=True, max_length=42)


tokenized_datasets_train = dataset_train_eval['train'].map(tokenize_function, batched=True)
tokenized_datasets_eval = dataset_train_eval['test'].map(tokenize_function, batched=True)

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

In [169]:
# We load the model, and specify the num_labels as 3 [pos, neg, neutral]
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [175]:
# Evaluation metrics, we can include more such as: F-measure
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

In [ ]:
# import pytorch and check for GPU availability, for using torch.argmax in the future
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

In [192]:
# Convert predictions to logits and use argmax function, need to change np.argmax to torch.argmax

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predictions, references=labels, average = 'micro')["accuracy"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average = 'micro')["recall"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall}

In [193]:
# Hyperparameter training, for now using default hyperparameters

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [194]:
# We can use optuna to do hyperparameter search
# def optuna_hp_space(trial):
#    return {
#        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
#        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
#    }

In [195]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

In [196]:
trainer.train()

/Users/parkhiagarwal/opt/anaconda3/envs/info/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.480195,0.855556
2,No log,0.561143,0.855556
3,0.448600,0.419766,0.830556


TrainOutput(global_step=630, training_loss=0.42635391477554563, metrics={'train_runtime': 1143.4946, 'train_samples_per_second': 4.408, 'train_steps_per_second': 0.551, 'total_flos': 108780953641920.0, 'train_loss': 0.42635391477554563, 'epoch': 3.0})

In [219]:
# Change to train_test_eval split and use it here, trainer.predict keeps giving an error
# trainer.predict(test_dataset = dataset_train_eval['test'])


TypeError: transformers.trainer.Trainer.predict() argument after ** must be a mapping, not Dataset